In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

# For metrics and splitting
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import ParameterGrid

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [3]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout=0.0):
        super(MLP, self).__init__()
        layers = []
        prev_dim = input_dim
        
        for i in range(n_layers):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        # final output layer
        layers.append(nn.Linear(prev_dim, 1))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        # x shape: (batch_size, input_dim)
        return self.net(x)  # shape: (batch_size, 1)

In [4]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout=0.0):
        super(RNN, self).__init__()
        # We'll use simple RNN
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.RNN(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            num_layers=n_layers, 
            batch_first=True,
            dropout=dropout if n_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # x shape: (batch_size, seq_len=1, input_dim)
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=x.device)

        out, _ = self.rnn(x, h0) 
        # out shape: (batch_size, seq_len, hidden_dim)
        out = out[:, -1, :]   # take last time step
        out = self.fc(out)    # shape: (batch_size, 1)
        return out

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout=0.0):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            num_layers=n_layers, 
            batch_first=True,
            dropout=dropout if n_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # x shape: (batch_size, seq_len=1, input_dim)
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        # out shape: (batch_size, seq_len, hidden_dim)
        out = out[:, -1, :]
        out = self.fc(out) 
        return out

In [6]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout=0.0):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(
            input_size=input_dim, 
            hidden_size=hidden_dim, 
            num_layers=n_layers, 
            batch_first=True,
            dropout=dropout if n_layers>1 else 0.0
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim, device=x.device)

        out, hn = self.gru(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [7]:
def build_dl_model(model_name, input_dim, params):
    """
    Builds a PyTorch model instance given model_name and hyperparams.
    model_name: 'MLP', 'RNN', 'LSTM', 'GRU'
    params:
      - 'hidden_dim': dimension of hidden layer(s)
      - 'n_layers': number of layers (2,3,4)
      - 'dropout': dropout rate
      ...
    """
    hidden_dim = params.get("hidden_dim", 64)
    n_layers   = params.get("n_layers", 2)
    dropout    = params.get("dropout", 0.0)
    
    if model_name == "MLP":
        model = MLP(input_dim, hidden_dim, n_layers, dropout=dropout)
    elif model_name == "RNN":
        model = RNN(input_dim, hidden_dim, n_layers, dropout=dropout)
    elif model_name == "LSTM":
        model = LSTM(input_dim, hidden_dim, n_layers, dropout=dropout)
    elif model_name == "GRU":
        model = GRU(input_dim, hidden_dim, n_layers, dropout=dropout)
    else:
        raise ValueError(f"Unknown model type: {model_name}")
    
    return model.to(device)

In [8]:
def sliding_window_forecast_with_torch(
    df,
    target_col,
    model_name,
    model_params,
    window_size=25,
    test_ratio=0.2,
    drop_cols=None,
    log_dir="model_dl_logs",
    log_filename="prediction_log.csv"
):
    """
    Rolls a window of size window_size through the data.
    For each window:
      - split train/test by test_ratio
      - standard scale X and y
      - train a PyTorch model
      - predict and inverse scale
      - log metrics (MSE, MAPE, R²)
    """
    if drop_cols is None:
        drop_cols = []
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_path = os.path.join(log_dir, log_filename)
    log_entries = []

    df = df.dropna().sort_index()
    n = len(df)
    
    for start in range(0, n - window_size + 1):
        window_df = df.iloc[start:start + window_size].copy()
        X_window = window_df.drop(columns=[target_col] + drop_cols, errors='ignore')
        y_window = window_df[target_col].values

        train_size = int(len(X_window) * (1 - test_ratio))
        if train_size < 1 or train_size >= len(X_window):
            continue
        
        X_train = X_window.iloc[:train_size].values
        X_test  = X_window.iloc[train_size:].values
        y_train = y_window[:train_size]
        y_test  = y_window[train_size:]

        # Scale features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled  = scaler_X.transform(X_test)

        # Scale target
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        y_test_scaled  = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

        # Format for model
        input_dim = X_train_scaled.shape[1]
        if model_name in ["RNN", "LSTM", "GRU"]:
            X_train_scaled = np.expand_dims(X_train_scaled, axis=1)
            X_test_scaled = np.expand_dims(X_test_scaled, axis=1)

        X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
        y_train_t = torch.tensor(y_train_scaled, dtype=torch.float32).view(-1, 1).to(device)
        X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
        y_test_t = torch.tensor(y_test_scaled, dtype=torch.float32).view(-1, 1).to(device)

        # Build and train model
        model = build_dl_model(model_name, input_dim, model_params)
        lr = model_params.get("learning_rate", 1e-3)
        optimizer = model_params.get("optimizer", "adam")
        epochs = model_params.get("epochs", 20)
        batch_size = model_params.get("batch_size", 32)

        if optimizer.lower() == "adam":
            opt = optim.Adam(model.parameters(), lr=lr)
        else:
            opt = optim.Adam(model.parameters(), lr=lr)

        criterion = nn.MSELoss()
        dataset_size = X_train_t.shape[0]
        num_batches = (dataset_size + batch_size - 1) // batch_size
        model.train()

        for ep in range(epochs):
            perm = torch.randperm(dataset_size)
            X_train_t = X_train_t[perm]
            y_train_t = y_train_t[perm]

            for b_idx in range(num_batches):
                start_b = b_idx * batch_size
                end_b = min(start_b + batch_size, dataset_size)
                xb = X_train_t[start_b:end_b]
                yb = y_train_t[start_b:end_b]

                opt.zero_grad()
                preds = model(xb)
                loss = criterion(preds, yb)
                loss.backward()
                opt.step()

        # Inference
        model.eval()
        with torch.no_grad():
            preds_test = model(X_test_t)
        preds_test = preds_test.cpu().numpy().ravel()
        y_true_test = y_test_t.cpu().numpy().ravel()

        # Inverse transform
        preds_test = scaler_y.inverse_transform(preds_test.reshape(-1, 1)).ravel()
        y_true_test = scaler_y.inverse_transform(y_true_test.reshape(-1, 1)).ravel()

        # Guard against invalid output
        if np.any(np.isnan(preds_test)) or np.any(np.isinf(preds_test)):
            print("⚠️ Skipping window due to NaN/inf in predictions.")
            continue

        mse = mean_squared_error(y_true_test, preds_test)
        mape = mean_absolute_percentage_error(y_true_test, preds_test)
        r2 = r2_score(y_true_test, preds_test)

        log_entry = {
            "model_name": model_name,
            "model_hyperparameters_dict": json.dumps(model_params),
            "window_size": window_size,
            "test_ratio": test_ratio,
            "start_date": str(df.index[start]),
            "end_date": str(df.index[start + window_size - 1]),
            "test_data_values_list": y_true_test.tolist(),
            "test_data_model_predictions_list": preds_test.tolist(),
            "MSE_score": mse,
            "MAPE_score": mape,
            "R^2_score": r2
        }
        log_entries.append(log_entry)

    log_df = pd.DataFrame(log_entries)
    if os.path.exists(log_path):
        existing = pd.read_csv(log_path)
        log_df = pd.concat([existing, log_df], ignore_index=True)
    log_df.to_csv(log_path, index=False)
    return log_df

In [9]:
def tune_torch_model(
    df,
    target_col,
    model_type,
    param_grid,
    window_sizes=[25, 50],
    test_ratio=0.2,
    drop_cols=None,
    log_dir="model_dl_logs"
):
    """
    Similar approach as classical ML tune_model. 
    We'll iterate over param_grid, run sliding_window_forecast_with_torch, 
    log results, average, and return summary.
    """
    if drop_cols is None:
        drop_cols = []
    
    param_list = list(ParameterGrid(param_grid))
    total_iterations = len(param_list) * len(window_sizes)
    pbar = tqdm(total=total_iterations, desc=f"Tuning {model_type}")
    
    summaries = []
    for w in window_sizes:
        for params in param_list:
            log_df = sliding_window_forecast_with_torch(
                df=df,
                target_col=target_col,
                model_name=model_type,
                model_params=params,
                window_size=w,
                test_ratio=test_ratio,
                drop_cols=drop_cols,
                log_dir=log_dir,
                log_filename=f"{model_type}_window_{w}.csv"
            )
            
            pbar.update(1)
            if log_df.empty:
                continue

            avg_mse  = log_df["MSE_score"].mean()
            avg_mape = log_df["MAPE_score"].mean()
            avg_r2   = log_df["R^2_score"].mean()

            summary_entry = {
                "model_name": model_type,
                "model_hyperparameters_dict": json.dumps(params),
                "window_size": w,
                "test_ratio": test_ratio,
                "avg_MSE": avg_mse,
                "avg_MAPE": avg_mape,
                "avg_R^2": avg_r2
            }
            summaries.append(summary_entry)
    pbar.close()
    
    if summaries:
        return pd.DataFrame(summaries)
    else:
        return pd.DataFrame()

In [10]:
def combine_and_top_logs(log_dir="model_dl_logs", tops=5):
    """
    Reads all CSV files in the given log directory, 
    groups by model_name and hyperparameters, 
    picks top "tops" by MAPE ascending for each model.
    """
    all_files = [f for f in os.listdir(log_dir) if f.endswith(".csv")]
    if not all_files:
        print(f"No CSV log files in {log_dir} directory.")
        return pd.DataFrame()
    
    dfs = [pd.read_csv(os.path.join(log_dir, f)) for f in all_files]
    combined_logs = pd.concat(dfs, ignore_index=True)
    
    grouped_summary = combined_logs.groupby(
        ["model_name","model_hyperparameters_dict"]
    )[["MSE_score","MAPE_score","R^2_score"]].mean().reset_index()

    top_n_list = []
    for model in grouped_summary["model_name"].unique():
        sub = grouped_summary[grouped_summary["model_name"] == model]
        top_n = sub.sort_values("MAPE_score", ascending=True).head(tops)
        top_n_list.append(top_n)
    top_n_combined = pd.concat(top_n_list, ignore_index=True)
    return top_n_combined

In [11]:
param_grids_torch = {
    "MLP": {
        "hidden_dim":    [32, 64, 128],
        "n_layers":      [1, 2, 3],
        "dropout":       [0.0, 0.2, 0.4],                # Regularization
        "learning_rate": [1e-2, 1e-3, 1e-4],             # Wider range
        "optimizer":     ["adam"],
        "epochs":        [25, 50, 100],                  # Reasonable ranges
        "batch_size":    [16, 32, 64],
        "weight_decay":  [0.0, 1e-4, 1e-3],              # L2 regularization
    },

    "RNN": {
        "hidden_dim":    [32, 64, 128],
        "n_layers":      [1, 2, 3],
        "dropout":       [0.0, 0.2, 0.4], 
        "learning_rate": [1e-2, 1e-3, 1e-4],
        "optimizer":     ["adam"],
        "epochs":        [25, 50, 100],
        "batch_size":    [16, 32, 64],
        "weight_decay":  [0.0, 1e-4, 1e-3]
    },

    "LSTM": {
        "hidden_dim":    [64, 128, 256],
        "n_layers":      [1, 2, 3],
        "dropout":       [0.0, 0.2, 0.4], 
        "learning_rate": [1e-2, 1e-3, 1e-4],
        "optimizer":     ["adam"],
        "epochs":        [25, 50, 100],
        "batch_size":    [16, 32, 64],
        "weight_decay":  [0.0, 1e-4, 1e-3]
    },

    "GRU": {
        "hidden_dim":    [64, 128, 256],
        "n_layers":      [1, 2, 3],
        "dropout":       [0.0, 0.2, 0.4], 
        "learning_rate": [1e-2, 1e-3, 1e-4],
        "optimizer":     ["adam"],
        "epochs":        [25, 50, 100],
        "batch_size":    [16, 32, 64],
        "weight_decay":  [0.0, 1e-4, 1e-3]
    }
}

In [12]:
with open("temp_output/quarterly_X_y.pkl", "rb") as f:
    data_q = pickle.load(f)

drop_columns = []
print(data_q.keys())  # Should show all the tickers like AAPL, MSFT, etc.
quaterly_data = data_q["AAPL"]

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO'])


In [13]:
# MLP on quarterly
mlp_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="MLP",
    param_grid=param_grids_torch["MLP"],
    window_sizes=[10],  # or [25,50]
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# RNN on quarterly
rnn_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="RNN",
    param_grid=param_grids_torch["RNN"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# LSTM on quarterly
lstm_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="LSTM",
    param_grid=param_grids_torch["LSTM"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

# GRU on quarterly
gru_summary_q = tune_torch_model(
    df=quaterly_data,
    target_col="y",
    model_type="GRU",
    param_grid=param_grids_torch["GRU"],
    window_sizes=[10],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_q"
)

Tuning GRU: 100%|██████████| 2187/2187 [3:07:10<00:00,  5.14s/it]  


In [15]:
combined_summary_q = pd.concat([mlp_summary_q, rnn_summary_q, lstm_summary_q, gru_summary_q], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_q)

# Alternatively, read all prediction log files from log directory
top5_summary_q = combine_and_top_logs(log_dir="model_dl_logs_q")
print("Top 5 Configurations per Model:")
display(top5_summary_q)

Combined Tuning Summary:


,model_name,model_hyperparameters_dict,window_size,test_ratio,avg_MSE,avg_MAPE,avg_R^2
0,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,778.982207,0.177793,-183.826692
1,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,769.949728,0.176999,-200.679550
2,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,835.726052,0.181807,-198.160551
3,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,798.616589,0.180312,-177.330999
4,MLP,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",10,0.2,787.044281,0.179981,-173.335006
...,...,...,...,...,...,...,...
8743,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.831696,0.183870,-220.677119
8744,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.741247,0.183856,-220.627070
8745,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.653085,0.183842,-220.581051
8746,GRU,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 1...",10,0.2,816.567877,0.183829,-220.532425


Top 5 Configurations per Model:


,model_name,model_hyperparameters_dict,MSE_score,MAPE_score,R^2_score
0,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",604.429245,0.148500,-114.641782
1,GRU,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 2...",624.912527,0.149950,-118.385994
2,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",638.081727,0.150358,-134.695980
3,GRU,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 1...",604.974328,0.150906,-120.426897
4,GRU,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 1...",599.377000,0.150942,-108.714437
5,LSTM,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",592.196254,0.145152,-73.316614
6,LSTM,"{""batch_size"": 32, ""dropout"": 0.4, ""epochs"": 2...",603.333849,0.147617,-169.880725
7,LSTM,"{""batch_size"": 16, ""dropout"": 0.0, ""epochs"": 2...",592.335330,0.148016,-82.748094
8,LSTM,"{""batch_size"": 16, ""dropout"": 0.4, ""epochs"": 2...",602.140783,0.148135,-153.402797
9,LSTM,"{""batch_size"": 64, ""dropout"": 0.4, ""epochs"": 2...",589.238387,0.148434,-160.610464


In [16]:
with open("temp_output/daily_X_y.pkl", "rb") as f:
    data_d = pickle.load(f)
    
drop_columns = []
print(data_d.keys())  # Should show all the tickers like AAPL, MSFT, etc.
daily_data = data_d["AAPL"]
daily_data = daily_data.iloc[-128:,:]
daily_data

dict_keys(['AAPL', 'MSFT', 'LLY', 'UNH', 'V', 'MA', 'GOOGL', 'META', 'AMZN', 'TSLA', 'PG', 'WMT', 'RTX', 'UNP', 'XOM', 'CVX', 'LIN', 'SHW', 'AMT', 'PLD', 'NEE', 'SO', '^GSPC'])


,Adj Close,Close,High,Low,Open,Volume,SMA5,SMA50,SMA200,MACDLine,...,^IXIC,^DJI,^VIX,CL=F,GC=F,SI=F,^TNX,DX-Y.NYB,FedFundsRate,y
2024-07-01,216.023956,216.750000,217.509995,211.919998,212.089996,60402900,212.045334,190.963962,182.925126,6.332360,...,17879.300781,39169.519531,12.22,83.379997,2327.600098,29.299999,4.479,105.900002,5.33,219.532166
2024-07-02,219.532166,220.270004,220.380005,215.100006,216.149994,58046200,214.277829,192.070114,183.150480,6.644230,...,18028.759766,39331.851562,12.03,82.809998,2323.000000,29.353001,4.436,105.720001,5.33,220.807892
2024-07-03,220.807892,221.550003,221.550003,219.029999,220.000000,37369800,215.932269,193.185060,183.385837,6.914623,...,18188.300781,39308.000000,12.09,83.879997,2359.800049,30.548000,4.355,105.400002,5.33,225.581833
2024-07-05,225.581833,226.339996,226.449997,221.649994,221.649994,60412400,218.372064,194.374384,183.630370,7.428497,...,18352.759766,39375.871094,12.48,83.160004,2388.500000,31.388000,4.272,104.879997,5.33,227.056885
2024-07-08,227.056885,227.820007,227.850006,223.250000,227.089996,59085900,221.800546,195.551008,183.876819,7.864117,...,18403.740234,39344.789062,12.37,82.330002,2355.199951,30.618000,4.269,105.010002,5.33,227.913986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,257.916443,258.200012,258.209991,255.289993,255.490005,23234700,252.881967,235.564016,211.182570,6.067327,...,20031.130859,43297.031250,14.27,70.099998,2620.000000,29.974001,4.591,108.260002,4.33,258.735504
2024-12-26,258.735504,259.019989,260.100006,257.630005,258.190002,27237100,255.073553,236.071997,211.614168,6.300019,...,20020.359375,43325.800781,14.73,69.620003,2638.800049,30.047001,4.579,108.129997,4.33,255.309296
2024-12-27,255.309296,255.589996,258.700012,253.059998,257.829987,42355300,256.232281,236.552763,212.039086,6.137217,...,19722.029297,42992.210938,15.95,70.599998,2617.199951,29.655001,4.619,108.000000,4.33,251.923019
2024-12-30,251.923019,252.199997,253.500000,250.750000,252.229996,35557500,255.774783,236.958419,212.437766,5.669595,...,19486.789062,42573.730469,17.40,70.989998,2606.100098,29.106001,4.545,108.129997,4.33,250.144974


In [ ]:
# MLP
mlp_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="MLP",
    param_grid=param_grids_torch["MLP"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# RNN
rnn_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="RNN",
    param_grid=param_grids_torch["RNN"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# LSTM
lstm_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="LSTM",
    param_grid=param_grids_torch["LSTM"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

# GRU
gru_summary_d = tune_torch_model(
    df=daily_data,
    target_col="y",
    model_type="GRU",
    param_grid=param_grids_torch["GRU"],
    window_sizes=[25],
    test_ratio=0.2,
    drop_cols=drop_columns,
    log_dir="model_dl_logs_d"
)

Tuning MLP:  19%|█▊        | 410/2187 [2:00:23<15:03:59, 30.52s/it]

In [ ]:
combined_summary_d = pd.concat([mlp_summary_d, rnn_summary_d, lstm_summary_d, gru_summary_d], ignore_index=True)
print("Combined Tuning Summary:")
display(combined_summary_d)

# Alternatively, read all prediction log files from log directory
top5_summary_d = combine_and_top_logs(log_dir="model_dl_logs_d")
print("Top 5 Configurations per Model:")
display(top5_summary_d)

In [ ]:
top_summary_q = combine_and_top_logs(log_dir="model_dl_logs_q", tops=25)
for d in top_summary_q['model_hyperparameters_dict']:
    print(json.loads(d))
    
print("\n"+"="*100+"\n")

top_summary_d = combine_and_top_logs(log_dir="model_dl_logs_d", tops=25)
for d in top_summary_d['model_hyperparameters_dict']:
    print(json.loads(d))

{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.04, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 4800}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 6400}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 8000}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 3200}
{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 1600}
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 1600}
{'learni